# IESB - Miner II - Aula 08 - Classes desbalanceadas

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Parte 1
## Tratamento de dados e execução do modelo

In [ ]:
# Importando os dados
df = pd.read_csv('/kaggle/input/bank-marketing/bank-additional-full.csv', sep=';')

df.shape

In [ ]:
# Verificando quantidades e tipos
df.info()

In [ ]:
# Visualizando os dados
df.head().T

In [ ]:
# Não temos valores nulos mas temos dados como 'unknown'
# Para facilitar vamos considerar esses dados como um tipo específico
df['job'].value_counts()

In [ ]:
# Verificando também a coluna 'loan'
df['loan'].value_counts()

In [ ]:
# O problema da variável 'duration' - data leak - vazamento de dados
df.drop(columns=['duration'], axis=1, inplace=True)

In [ ]:
# Convertendo as colunas object para colunas categóricas
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].astype('category').cat.codes
        
df.info()

In [ ]:
# Separando o dataframe
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.20, random_state=42)

train.shape, test.shape

In [ ]:
# Lista das colunas a serem usadas para treino
feats = [c for c in df.columns if c not in ['y']]

In [ ]:
# Importando o RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

# Instanciando um objeto RandomForest
rf = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)

# Treinando o modelo
rf.fit(train[feats], train['y'])

In [ ]:
# Fazer previsões usando o modelo treinado
preds = rf.predict(test[feats])

# Importando a métrica
from sklearn.metrics import accuracy_score

# Avaliando o modelo com relação aos dados de teste
accuracy_score(test['y'], preds)

In [ ]:
# Verificando a distribuição da variável target
df['y'].value_counts()

In [ ]:
# Verificando a distribuição da variável target (%)
df['y'].value_counts(normalize=True)

In [ ]:
# Verificando a distribuição da variável target (%) - Base de treino
train['y'].value_counts(normalize=True)

In [ ]:
# Verificando a distribuição da variável target (%) - Base de teste
test['y'].value_counts(normalize=True)

## Confusion Matrix

In [ ]:
# Importando uma biblioteca específica para plotar gráficos relacionada
# com o scikitlearn
import scikitplot as skplt

# Matriz de Confusão - Dados de teste
skplt.metrics.plot_confusion_matrix(test['y'], preds)

# Parte 2
## Tratando classes desbalanceadas

## Fazendo Random Under-Sampling e Random Over-Sampling
Vamos usar o própio sklearn para realizar under e over sampling de forma aleatória

In [ ]:
# Importando a biblioteca
from sklearn.utils import resample

# Separando os dados de acordo com a classificação
df_no = df[df['y'] == 0]
df_yes = df[df['y'] == 1]

df_no.shape, df_yes.shape

### Random Over-Sampling 

In [ ]:
# Over-Sampling
df_yes_over = resample(df_yes, # vamos aumentar a classe menor
                       replace=True, # sample com replacement
                       n_samples=len(df_no), # igualando a maior classe
                       random_state=42)

# juntando os dados
df_over= pd.concat([df_no, df_yes_over])

# check new class counts
df_over['y'].value_counts()

In [ ]:
# Executando o modelo com df_over

# Dividindo em treino e teste
train, test = train_test_split(df_over, test_size=0.2, random_state=42)

# Treinar o modelo
rf.fit(train[feats], train['y'])

# Previsões na base de teste
preds_test = rf.predict(test[feats])

# Medir a acurácia
accuracy_score(test['y'], preds_test)

In [ ]:
# Plotando a matriz de confusao para os dados de teste
skplt.metrics.plot_confusion_matrix(test['y'], preds_test)

### Random Under-Sampling

In [ ]:
# Under-Sampling
df_no_under = resample(df_no, # vamos diminuir a classe maior
                       replace=False, # sample sem replacement
                       n_samples=len(df_yes), # igualando a menor classe
                       random_state=42)

# juntando os dados
df_under= pd.concat([df_no_under, df_yes])

# check new class counts
df_under['y'].value_counts()

In [ ]:
# Executando o modelo com df_under

# Dividindo em treino e teste
train, test = train_test_split(df_under, test_size=0.2, random_state=42)

# Treinar o modelo
rf.fit(train[feats], train['y'])

# Previsões na base de teste
preds_test = rf.predict(test[feats])

# Medir a acurácia
accuracy_score(test['y'], preds_test)

In [ ]:
# Plotando a matriz de confusao para os dados de teste
skplt.metrics.plot_confusion_matrix(test['y'], preds_test)

## Usando a bilioteca imbalanced-learn
Essa biblioteca implementa diversos modelos diferentes para tratar classes desabalanceadas

In [ ]:
# Importando a biblioteca
import imblearn

In [ ]:
# Separando os dados de entrada e o target
X, y = df[feats], df[['y']]

### Imblearn Random Over-Sampling

In [ ]:
# Importando a biblioteca
from imblearn.over_sampling import RandomOverSampler

# Fazendo o over-sampling
ros = RandomOverSampler(random_state=42)
X_ros,y_ros= ros.fit_resample(X,y)

# Verificando os dados
y_ros['y'].value_counts()

In [ ]:
# Executando o modelo com imblearn over-sampling

# Juntando os dados
df_over = pd.concat([X_ros, y_ros], axis=1)

# Dividindo em treino e teste
train, test = train_test_split(df_over, test_size=0.2, random_state=42)

# Treinar o modelo
rf.fit(train[feats], train['y'])

# Previsões na base de teste
preds_test = rf.predict(test[feats])

# Medir a acurácia
accuracy_score(test['y'], preds_test)

In [ ]:
# Plotando a matriz de confusao para os dados de teste
skplt.metrics.plot_confusion_matrix(test['y'], preds_test)

### Imblearn Tomek-links (under-sampling)

In [ ]:
# Importando a biblioteca
from imblearn.under_sampling import TomekLinks

# Fazendo o under-sampling
tl = TomekLinks()
X_tl, y_tl = tl.fit_resample(X,y)

# Verificando os dados
y_tl['y'].value_counts()

In [ ]:
# Executando o modelo com Tomek-links

# Juntando os dados
df_under = pd.concat([X_tl, y_tl], axis=1)

# Dividindo em treino e teste
train, test = train_test_split(df_under, test_size=0.2, random_state=42)

# Treinar o modelo
rf.fit(train[feats], train['y'])

# Previsões na base de teste
preds_test = rf.predict(test[feats])

# Medir a acurácia
accuracy_score(test['y'], preds_test)

In [ ]:
# Plotando a matriz de confusao para os dados de teste
skplt.metrics.plot_confusion_matrix(test['y'], preds_test)

### Imblearn SMOTE (over-sampling)

In [ ]:
# Importando a biblioteca
from imblearn.over_sampling import SMOTE

# Fazendo o under-sampling
sm = SMOTE()
X_sm, y_sm = sm.fit_resample(X,y)

# Verificando os dados
y_sm['y'].value_counts()

In [ ]:
# Executando o modelo com SMOTE

# Juntando os dados
df_over = pd.concat([X_sm, y_sm], axis=1)

# Dividindo em treino e teste
train, test = train_test_split(df_over, test_size=0.2, random_state=42)

# Treinar o modelo
rf.fit(train[feats], train['y'])

# Previsões na base de teste
preds_test = rf.predict(test[feats])

# Medir a acurácia
accuracy_score(test['y'], preds_test)

In [ ]:
# Plotando a matriz de confusao para os dados de teste
skplt.metrics.plot_confusion_matrix(test['y'], preds_test)